# Debug Experiments Configuration

This notebook sets up debug experiments across multiple datasets (CIFAR-100, GTSRB, ImageNette) with various poisoning attacks.

In [ ]:
import torch
from pathlib import Path
from config.experiment_config import create_config
from utils.logging import setup_logging, get_logger

# Initialize logging
setup_logging()
logger = get_logger(__name__)

## Checkpoint Configuration

First, let's set up checkpoint handling:

In [ ]:
def find_checkpoint(model_type='wideresnet'):
    """Find the best or latest checkpoint for a model."""
    checkpoint_dir = Path('~/Notebooks/classify/checkpoints') / model_type
    checkpoint_dir = checkpoint_dir.expanduser()
    
    if not checkpoint_dir.exists():
        logger.warning(f"Checkpoint directory {checkpoint_dir} does not exist")
        return None
        
    # First try to find best checkpoint
    best_checkpoint = checkpoint_dir / 'wideresnet_best.pt'
    if best_checkpoint.exists():
        logger.info(f"Found best checkpoint: {best_checkpoint}")
        return best_checkpoint
        
    # Otherwise get latest checkpoint
    latest_checkpoint = checkpoint_dir / 'wideresnet_latest.pt'
    if latest_checkpoint.exists():
        logger.info(f"Found latest checkpoint: {latest_checkpoint}")
        return latest_checkpoint
        
    logger.warning(f"No checkpoints found in {checkpoint_dir}")
    return None

checkpoint_path = find_checkpoint()
if checkpoint_path:
    logger.info(f"Using checkpoint: {checkpoint_path}")
else:
    logger.warning("No checkpoint found, will train from scratch")

## Hardware Configuration

Let's check available hardware and set up device-specific configurations:

In [ ]:
# Determine hardware configuration
if torch.cuda.is_available():
    device_info = f"CUDA (GPU: {torch.cuda.get_device_name(0)})"
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    gpu_ids = [0]
elif torch.backends.mps.is_available():
    device_info = "MPS (Apple Silicon)"
    gpu_ids = []
else:
    device_info = "CPU"
    gpu_ids = []

print(f"Running on: {device_info}")

# Set hardware-dependent parameters
num_workers = 4 if not torch.backends.mps.is_available() else 0

## Create Base Configurations

Now let's create configurations for each dataset:

In [ ]:
# Common configuration for all datasets
base_config = {
    'model': {
        'name': 'wrn-28-10',
        'depth': 28,
        'widen_factor': 10
    },
    'training': {
        'batch_size': 128,
        'num_workers': num_workers,
        'pin_memory': True
    },
    'checkpoint': {
        'save_dir': str(checkpoint_path.parent) if checkpoint_path else 'checkpoints',
        'resume': True if checkpoint_path else False
    },
    'execution': {
        'max_workers': 1,
        'gpu_ids': gpu_ids
    },
    'output': {
        'base_dir': 'results',
        'save_models': True,
        'save_frequency': 10,
        'consolidated_file': 'debug_results.csv',
        'save_individual_results': True
    }
}

# Create dataset-specific configurations
cifar_config = create_config(
    'cifar100',
    **base_config,
    training={'epochs': 200},
    poison={
        'poison_type': 'ga',
        'poison_ratio': 0.1,
        'batch_size': 32,
        'ga_steps': 50,
        'ga_iterations': 100,
        'ga_lr': 0.1
    }
)

gtsrb_config = create_config(
    'gtsrb',
    **base_config,
    training={'epochs': 10},
    poison={
        'poison_type': 'pgd',
        'poison_ratio': 0.1,
        'batch_size': 32,
        'pgd_eps': 0.3,
        'pgd_alpha': 0.01,
        'pgd_steps': 40
    }
)

imagenette_config = create_config(
    'imagenette',
    **base_config,
    training={
        'epochs': 10,
        'batch_size': 64  # Smaller batch size for ImageNette
    },
    poison={
        'poison_type': 'pgd',
        'poison_ratio': 0.1,
        'batch_size': 32,
        'pgd_eps': 0.3,
        'pgd_alpha': 0.01,
        'pgd_steps': 40
    }
)

print("Configurations created for all datasets")

## Create Experiment Groups

Now let's set up the experiment groups for comparing different attacks:

In [ ]:
experiment_groups = {
    'basic_comparison': {
        'description': 'Basic comparison of attacks across different datasets',
        'experiments': [
            {
                'name': 'cifar100_debug',
                'dataset': 'cifar100',
                'attacks': ['ga', 'label_flip']
            },
            {
                'name': 'gtsrb_debug',
                'dataset': 'gtsrb',
                'attacks': ['pgd', 'ga', 'label_flip']
            },
            {
                'name': 'imagenette_debug',
                'dataset': 'imagenette',
                'attacks': ['pgd', 'ga', 'label_flip']
            }
        ]
    }
}

# Create final configuration with experiment groups
final_config = create_config(
    'cifar100',  # Base dataset
    **base_config,
    experiment_groups=experiment_groups
)

# Save the configuration
config_path = 'experiments/debug_config.yaml'
final_config.save_yaml(config_path)
print(f"Saved configuration to {config_path}")

## Run Experiments

Finally, let's run our experiments:

In [ ]:
from run_experiments import ExperimentManager

# Create and run experiment manager
manager = ExperimentManager(config_path)
print(f"Total experiments to run: {manager.total_experiments}")

# Run experiments
try:
    manager.run_experiments()
except Exception as e:
    logger.error(f"Error during experiments: {str(e)}")
    raise